In [1]:
import os


from lm_polygraph.stat_calculators import *
from lm_polygraph.utils.openai_chat import OpenAIChat
from transformers import AutoTokenizer
from lm_polygraph.model_adapters import WhiteBoxModelBasic
from torch.utils.data import DataLoader

calc_claim_extractor = ClaimsExtractor(OpenAIChat("gpt-3.5-turbo"))

model = None
tokenizer = AutoTokenizer.from_pretrained("core42/jais-13b", trust_remote_code=True)
model_adapter = WhiteBoxModelBasic(model, tokenizer)



/home/boda/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/boda/miniforge3/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/boda/miniforge3/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/boda/miniforge3/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree

In [2]:
en_inputs = [    '''Albert Einstein was a renowned theoretical physicist who is best known for developing the theory of relativity, which revolutionized our understanding of time, space, and gravity. He was born on March 14, 1879, in Ulm, Germany. 
Einstein showed early signs of brilliance and independence, which often led to conflicts with authority figures. He studied physics at the Polytechnic Institute in Zurich and later worked as a patent clerk in Switzerland. In 1905, Einstein published a series of groundbreaking papers that laid the foundation for modern physics, including his famous equation E=mc^2, which describes the relationship between energy and mass.

Einstein's work on the theory of relativity earned him international fame and acclaim, and he was awarded the Nobel Prize in Physics in 1921. Throughout his life, he continued to make significant contributions to physics, including his work on quantum mechanics and the unified field theory.

Einstein was also a prominent advocate for civil rights, pacifism, and nuclear disarmament. He emigrated to the United States in 1933 to escape the rising tide of anti-Semitism in Europe and took up a position at the Institute for Advanced Study in Princeton, New Jersey. 

Albert Einstein passed away on April 18, 195''']

ar_inputs = [
" كان الجاحظ كاتباً وفيلسوفاً مسلماً عاش في بغداد خلال العصر العباسي. كان معروفاً بدهائه وذكائه ، وكان مؤلفاً غزير الإنتاج كتب في مجموعة متنوعة من الموضوعات بما في ذلك الأدب والفلسفة وعلم الكلام. كما عرف بأسلوبه الفريد في الكتابة الذي تضمن استخدام الاستعارات والتشبيهات."
]

In [3]:
data_loader = DataLoader(ar_inputs, batch_size=2, shuffle=False, collate_fn=lambda x: x)
for batch in data_loader:
    encoded = tokenizer(batch, padding=True, return_tensors="pt")
    deps = {"greedy_tokens": encoded.input_ids, "greedy_texts": batch}

    claims = calc_claim_extractor(deps, texts=batch, model=model_adapter,language="ar")


    print(claims)

{'claims': [[Claim(claim_text='الجاحظ كان كاتباً.', sentence=' كان الجاحظ كاتباً وفيلسوفاً مسلماً عاش في بغداد خلال العصر العباسي', aligned_tokens=[0, 1, 2, 3, 4]), Claim(claim_text='الجاحظ كان مسلماً.', sentence=' كان الجاحظ كاتباً وفيلسوفاً مسلماً عاش في بغداد خلال العصر العباسي', aligned_tokens=[1, 2, 10, 11]), Claim(claim_text='الجاحظ عاش في بغداد.', sentence=' كان الجاحظ كاتباً وفيلسوفاً مسلماً عاش في بغداد خلال العصر العباسي', aligned_tokens=[1, 2, 12, 13, 14]), Claim(claim_text='الجاحظ عاش خلال العصر العباسي.', sentence=' كان الجاحظ كاتباً وفيلسوفاً مسلماً عاش في بغداد خلال العصر العباسي', aligned_tokens=[1, 2, 12, 16, 17]), Claim(claim_text='كان مؤلفاً غزير الإنتاج.', sentence=' كان معروفاً بدهائه وذكائه ، وكان مؤلفاً غزير الإنتاج كتب في مجموعة متنوعة من الموضوعات بما في ذلك الأدب والفلسفة وعلم الكلام', aligned_tokens=[30, 31, 32, 33, 34, 35]), Claim(claim_text='كتب في الأدب والفلسفة وعلم الكلام.', sentence=' كان معروفاً بدهائه وذكائه ، وكان مؤلفاً غزير الإنتاج كتب في مجموعة مت

In [4]:
len(claims['claims'][0])

7

In [5]:
for c in claims['claims'][0]:
    print(c.claim_text)
    print(c.sentence)
    print(c.aligned_tokens)
    print('-' * 50)


الجاحظ كان كاتباً.
 كان الجاحظ كاتباً وفيلسوفاً مسلماً عاش في بغداد خلال العصر العباسي
[0, 1, 2, 3, 4]
--------------------------------------------------
الجاحظ كان مسلماً.
 كان الجاحظ كاتباً وفيلسوفاً مسلماً عاش في بغداد خلال العصر العباسي
[1, 2, 10, 11]
--------------------------------------------------
الجاحظ عاش في بغداد.
 كان الجاحظ كاتباً وفيلسوفاً مسلماً عاش في بغداد خلال العصر العباسي
[1, 2, 12, 13, 14]
--------------------------------------------------
الجاحظ عاش خلال العصر العباسي.
 كان الجاحظ كاتباً وفيلسوفاً مسلماً عاش في بغداد خلال العصر العباسي
[1, 2, 12, 16, 17]
--------------------------------------------------
كان مؤلفاً غزير الإنتاج.
 كان معروفاً بدهائه وذكائه ، وكان مؤلفاً غزير الإنتاج كتب في مجموعة متنوعة من الموضوعات بما في ذلك الأدب والفلسفة وعلم الكلام
[30, 31, 32, 33, 34, 35]
--------------------------------------------------
كتب في الأدب والفلسفة وعلم الكلام.
 كان معروفاً بدهائه وذكائه ، وكان مؤلفاً غزير الإنتاج كتب في مجموعة متنوعة من الموضوعات بما في ذلك الأد